In [44]:
import csv
import numpy as np
import pandas as pd
import datetime as dt

# extract fields
fields = ['job_id', 'owner', 'job_name', 'slots', 'mem', 'start_epoch', 'end_epoch', 'wait_time_seconds', 'io', 'group']
df = pd.read_csv('SelectedJobs/combined.csv', skipinitialspace=True, usecols=fields)

# save to
df.to_csv(r'newdata/1_selection_data.csv', index = False)

df


,job_id,owner,job_name,group,start_epoch,end_epoch,mem,io,slots,wait_time_seconds
0,1892644,aosmansk,QLOGIN,bio,1.593562e+09,1.593613e+09,0.000000e+00,0.0000,1.0,429.000
1,1892645,bjayee,job_name,chem,1.593566e+09,1.593566e+09,4.368552e+03,0.0057,16.0,3096.127
2,1892646,daray,QLOGIN,bio,1.593564e+09,1.593592e+09,0.000000e+00,0.0000,1.0,1.862
3,1892657,edgardga,r_bl,ME,1.593566e+09,1.593574e+09,3.262322e+06,7129.2990,32.0,229.884
4,1892658,tpeltola,tcad_test,phys,1.593566e+09,1.593635e+09,4.832008e+07,0.3959,36.0,110987.000
...,...,...,...,...,...,...,...,...,...,...
81096,1997849,bdankesr,lammps,ME,1.601526e+09,1.601539e+09,2.284000e-01,0.0456,216.0,18005.007
81097,1997850,bdankesr,lammps,ME,1.601526e+09,1.601540e+09,2.366000e-01,0.0228,216.0,17997.786
81098,1997851,bdankesr,lammps,ME,1.601526e+09,1.601540e+09,2.426000e-01,0.0429,216.0,17998.510
81099,1997852,lbunting,mpi,atms,1.601526e+09,1.601526e+09,2.112400e+00,0.1098,288.0,17769.754


#### Show difference in time

In [45]:
# remove the incorrect records ------------------

df = pd.read_csv("newdata/1_selection_data.csv", parse_dates=True)

df["io"] = df['io']

# Tq = log( start_time - submit_time )
df["queuetime"] = df['wait_time_seconds']

# Tr = log( end_time - start_time )
df["runtime"] = df['end_epoch'] - df['start_epoch']

# Remove negative records and convert 0 -> 0.[0]1 ------------------

df = df[(df.mem >= 0.0) & (df.slots >= 0.0) & (df.queuetime >= 0.0) & (df.runtime >= 0.0) & (df.io >= 0.0)]

df.to_csv(r'newdata/2_non_neg.csv', index = False)

print(df.columns.tolist())
list(df.columns.values)

['job_id', 'owner', 'job_name', 'group', 'start_epoch', 'end_epoch', 'mem', 'io', 'slots', 'wait_time_seconds', 'queuetime', 'runtime']


['job_id',
 'owner',
 'job_name',
 'group',
 'start_epoch',
 'end_epoch',
 'mem',
 'io',
 'slots',
 'wait_time_seconds',
 'queuetime',
 'runtime']

In [42]:
#  scaled with standard scaler
from pandas import read_csv
from pandas import DataFrame
from pandas.plotting import scatter_matrix
from sklearn.preprocessing import StandardScaler

df = pd.read_csv("newdata/2_non_neg.csv", parse_dates=True)

attr = ['slots', 'mem', 'queuetime', 'runtime', 'io']

df["slots"] = df["slots"] / 36

# retrieve just the numeric input values
data = df[attr].to_numpy()

# perform a robust scaler transform of the dataset
trans = StandardScaler()
data = trans.fit_transform(data)

# convert the array back to a dataframe
dataset = DataFrame(data)

features = ['N_n', 'M_u', 'T_q', 'T_r', 'I_o']

dataset.columns = features

dataset.to_csv(r'newdata/sscaled_data.csv', index = False)

#### Just data

In [76]:
# no scale whatsoever

from pandas import read_csv
from pandas import DataFrame
from pandas.plotting import scatter_matrix
from sklearn.preprocessing import StandardScaler
import random


df = pd.read_csv("newdata/2_non_neg.csv", parse_dates=True)

df["slots"] = df["slots"] / 36

df["g"] = df["group"] 

# df["group"] = df["group"] + str(random.randint(0, 100000))

# for index, row in df.iterrows():
#     row["g"] = str(index)

for index in df.index:
    df.loc[index,'g'] = df.loc[index,'g'] + str(index)
#     print(df.loc[index,'g'] + str(index))
#     df.loc[index,'g'] = df["group"] + str(index)
#     if df.loc[index,'Name']=='Mr. Elon R. Musk':
#         df.loc[index,'Title'] = 'The Boss Man'
#     elif df.loc[index,'Name']=='Mr. Zachary J. Kirkhorn':
#         df.loc[index,'Title'] = 'The Money Man'
#     else:
#         df.loc[index,'Title'] = 'Another Dude'

df

,job_id,owner,job_name,group,start_epoch,end_epoch,mem,io,slots,wait_time_seconds,queuetime,runtime,g
0,1892644,aosmansk,QLOGIN,bio,1.593562e+09,1.593613e+09,0.000000e+00,0.0000,0.027778,429.000,429.000,51057.0,bio0
1,1892645,bjayee,job_name,chem,1.593566e+09,1.593566e+09,4.368552e+03,0.0057,0.444444,3096.127,3096.127,22.0,chem1
2,1892646,daray,QLOGIN,bio,1.593564e+09,1.593592e+09,0.000000e+00,0.0000,0.027778,1.862,1.862,28237.0,bio2
3,1892657,edgardga,r_bl,ME,1.593566e+09,1.593574e+09,3.262322e+06,7129.2990,0.888889,229.884,229.884,7803.0,ME3
4,1892658,tpeltola,tcad_test,phys,1.593566e+09,1.593635e+09,4.832008e+07,0.3959,1.000000,110987.000,110987.000,68584.0,phys4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
81096,1997849,bdankesr,lammps,ME,1.601526e+09,1.601539e+09,2.284000e-01,0.0456,6.000000,18005.007,18005.007,13035.0,ME81096
81097,1997850,bdankesr,lammps,ME,1.601526e+09,1.601540e+09,2.366000e-01,0.0228,6.000000,17997.786,17997.786,14547.0,ME81097
81098,1997851,bdankesr,lammps,ME,1.601526e+09,1.601540e+09,2.426000e-01,0.0429,6.000000,17998.510,17998.510,14600.0,ME81098
81099,1997852,lbunting,mpi,atms,1.601526e+09,1.601526e+09,2.112400e+00,0.1098,8.000000,17769.754,17769.754,10.0,atms81099


In [77]:
attr = ['slots', 'mem', 'queuetime', 'runtime', 'io', 'g']

# retrieve just the numeric input values
data = df[attr].to_numpy()

# convert the array back to a dataframe
dataset = DataFrame(data)

features = ['N_n', 'M_u', 'T_q', 'T_r', 'I_o', 'name']

dataset.columns = features

dataset.to_csv(r'newdata/data.csv', index = False)

dataset

,N_n,M_u,T_q,T_r,I_o,name
0,0.0277778,0,429,51057,0,bio0
1,0.444444,4368.55,3096.13,22,0.0057,chem1
2,0.0277778,0,1.862,28237,0,bio2
3,0.888889,3.26232e+06,229.884,7803,7129.3,ME3
4,1,4.83201e+07,110987,68584,0.3959,phys4
...,...,...,...,...,...,...
81096,6,0.2284,18005,13035,0.0456,ME81096
81097,6,0.2366,17997.8,14547,0.0228,ME81097
81098,6,0.2426,17998.5,14600,0.0429,ME81098
81099,8,2.1124,17769.8,10,0.1098,atms81099


#### Log

In [39]:
# Scale with log

# read data
df = pd.read_csv("newdata/2_non_neg.csv", parse_dates=True)

df = df.replace([0],0.0000000001)

# Nn = log( nproc/Cn )
df["N_n"] = np.log(df["slots"]/36)

# Mu = log( mem_used )
df["M_u"] = np.log(df["mem"])

# Tq = log( start_time - submit_time )
df["T_q"] = np.log(df["queuetime"])

# Tr = log( end_time - start_time )
df["T_r"] = np.log(df["runtime"])

# I_o = log( io )
df["I_o"] = np.log(df["io"])


df.drop(['start_epoch', 'end_epoch', 'mem', 'slots', 'queuetime', 'runtime', 'io', 'wait_time_seconds'], inplace=True, axis=1)

df.to_csv(r'newdata/log_data.csv', index = False)

print(df.columns.tolist())


['job_id', 'owner', 'job_name', 'N_n', 'M_u', 'T_q', 'T_r', 'I_o']


#### Helper functions

In [ ]:
# find the lowest non-zero
df[df['mem'].gt(0)].min()

In [ ]:
# subtract date time
import datetime as dt

start = job_accounting["start_time"][0]
submit = job_accounting["submission_time"][0]
print(start, submit)

a = dt.datetime.strptime(start, '%Y-%m-%d %H:%M:%S')
b = dt.datetime.strptime(submit, '%Y-%m-%d %H:%M:%S')

print(a-b)

In [ ]:
import pandas as pd
# Smaller data

rows = 5000

df1 = pd.read_csv("data/2_non_neg.csv", skipinitialspace=True)
df1['id'] = df1['group'] + "_" + df1['_id'].str.slice(-5,-1)  
df1a = df1['id'].head(rows)

df2 = pd.read_csv("data/data_all.csv", skipinitialspace=True)
df2 = df2.head(rows)

df3 = pd.concat([df1a, df2], axis=1, ignore_index=True)
df3.columns = ['name', 'N_n', 'M_u', 'T_q', 'T_r']

df3.to_csv(r'data/data.csv', index = False)



In [ ]:
import pandas as pd

df = pd.read_csv('data/1_selection_data.csv', skipinitialspace=True)

df['_id'][1][-5:-1]
# pd.unique(df['owner']).size

In [ ]:
df = pd.read_csv('data/data.csv', skipinitialspace=True)

df[~df.name.str.contains('\w')].name.count()